In [2]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.service import Service
from webdriver_manager.chrome import ChromeDriverManager
from bs4 import BeautifulSoup
import time
import pandas as pd

service = Service(ChromeDriverManager().install())
options = webdriver.ChromeOptions()
options.add_argument("--disable-gpu")

driver = webdriver.Chrome(service=service, options=options)

# Ganti dengan URL halaman review toko di Tokopedia
url = "https://www.tokopedia.com/dkmall/review"

driver.get(url)

reviews = []

while True:
    # Ambil elemen review
    time.sleep(3)  # Tunggu halaman masuk
    soup = BeautifulSoup(driver.page_source, "html.parser")
    try:
        review_elements = soup.findAll("article", attrs={'class':'css-ccpe8t'})

        for review in review_elements:
            username = review.find("span", attrs={"class":"name"}).text
            comment = review.find("span", attrs={"data-testid":"lblItemUlasan"}).text
            product_container = review.findAll("a",attrs={"class":"styProduct"})

            for element in product_container:
                product = review.find("p", attrs={"data-unify":"Typography"}).text
                
                reviews.append({
                    "username": username,
                    "review": comment,
                    "product": product
                })
    except:
        print("there is error")
        
    # Cek apakah ada tombol "Next" untuk lanjut ke halaman berikutnya
    try:
        next_button = driver.find_element(By.CSS_SELECTOR, 'button[aria-label="Laman berikutnya"]')
        next_button.click()
        time.sleep(2)
    except:
        print("No more pages.")
        break  # Keluar jika tidak ada tombol "Next"

driver.quit()

# Simpan ke CSV
df = pd.DataFrame(reviews)
df.to_csv("tokopedia_review.csv", index=False)
print("Scraping selesai! Data disimpan dalam tokopedia_reviews.csv")

C:\Users\user\AppData\Local\Temp\ipykernel_3716\339290948.py:27: DeprecationWarning: Call to deprecated method findAll. (Replaced by find_all) -- Deprecated since version 4.0.0.
  review_elements = soup.findAll("article", attrs={'class':'css-ccpe8t'})
C:\Users\user\AppData\Local\Temp\ipykernel_3716\339290948.py:32: DeprecationWarning: Call to deprecated method findAll. (Replaced by find_all) -- Deprecated since version 4.0.0.
  product_container = review.findAll("a",attrs={"class":"styProduct"})


there is error
there is error
there is error
there is error
there is error
there is error
there is error
there is error
there is error
there is error
there is error
there is error
there is error
there is error
there is error
there is error
there is error
there is error
there is error
there is error
there is error
there is error
there is error
there is error
No more pages.
Scraping selesai! Data disimpan dalam tokopedia_reviews.csv


In [24]:
import pandas as pd
from Sastrawi.StopWordRemover.StopWordRemoverFactory import StopWordRemoverFactory
from Sastrawi.Stemmer.StemmerFactory import StemmerFactory
import re

# Load data hasil scraping
df = pd.read_csv("tokopedia_review.csv")

# Tampilkan contoh data
print("data dari csv: ")
print(df.head())

# Inisialisasi stopword remover & stemmer
stopword_factory = StopWordRemoverFactory()
stopword_remover = stopword_factory.create_stop_word_remover()
stemmer_factory = StemmerFactory()
stemmer = stemmer_factory.create_stemmer()

# Fungsi preprocessing
def clean_text(text):
    text = text.lower()  # Lowercase
    text = re.sub(r'[^a-zA-Z0-9\s]', '', text)  # Hapus karakter spesial
    text = stopword_remover.remove(text)  # Hapus stopword
    text = stemmer.stem(text)  # Stemming
    return text

# Terapkan ke data
df["clean_review"] = df["review"].astype(str).apply(clean_text)

slang_dict = {
    "yg": "yang",
    "bgt": "banget",
    "tdk": "tidak",
    "gk": "gak",
    "trs": "terus",
    "bnr2": "benar-benar",
    "kayu2": "kayu-kayu",
    "sukaaa": "sukaaa",
    "jd" : "jadi",
    "byk2" : "banyak",
    "mantaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaap": "mantap",
    "bsnahabbaabananananananananananananananannaannaananan":""
}

def normalize_slang(text):
    words = text.split()
    words = [slang_dict[word] if word in slang_dict else word for word in words]
    return " ".join(words)

df["clean_review"] = df["clean_review"].apply(normalize_slang)
print(df.head())

data dari csv: 
  username                                             review  \
0    ANDRI  mendarat dengan aman, packing aman, engga boco...   
1    K***s  thank you, hotto nya asli, pengiriman cepat, h...   
2   Awalia                                             mantap   
3    gugun                                               good   
4  Marissa                                     sesuai pesanan   

                                             product  
0  Autumn 7% Glycolic Acid Toner Solution Exfolia...  
1  Hotto Purto Multigrain Superfood Purple Potato...  
2  Azarine Calm My Acne Sunscreen Moisturiser SPF...  
3  Ms Glow Men Energizer Facial Wash / Face Wash ...  
4   Golden Viera Soap Original Asli Paket Hemat 3pcs  
  username                                             review  \
0    ANDRI  mendarat dengan aman, packing aman, engga boco...   
1    K***s  thank you, hotto nya asli, pengiriman cepat, h...   
2   Awalia                                             mantap   
3  

In [25]:
import spacy

nlp = spacy.load("xx_ent_wiki_sm")  # Model NER multibahasa

def extract_entities(text):
    doc = nlp(text)
    entities = [(ent.text, ent.label_) for ent in doc.ents]
    return entities

df["entities"] = df["clean_review"].astype(str).apply(extract_entities)
print(df.head())

  username                                             review  \
0    ANDRI  mendarat dengan aman, packing aman, engga boco...   
1    K***s  thank you, hotto nya asli, pengiriman cepat, h...   
2   Awalia                                             mantap   
3    gugun                                               good   
4  Marissa                                     sesuai pesanan   

                                             product  \
0  Autumn 7% Glycolic Acid Toner Solution Exfolia...   
1  Hotto Purto Multigrain Superfood Purple Potato...   
2  Azarine Calm My Acne Sunscreen Moisturiser SPF...   
3  Ms Glow Men Energizer Facial Wash / Face Wash ...   
4   Golden Viera Soap Original Asli Paket Hemat 3pcs   

                                        clean_review  \
0    darat aman packing aman engga bocor kirim cepat   
1  thank you hotto nya asli kirim cepat harga sah...   
2                                             mantap   
3                                               

In [27]:
import networkx as nx
from pyvis.network import Network

G = nx.Graph()

for index, row in df.iterrows():
    user = row["username"]
    review = row["clean_review"]
    entities = row["entities"]

    # Tambahkan node user
    G.add_node(user, type="User")

    # Tambahkan node entitas (Produk/Sentimen)
    for ent_text, ent_label in entities:
        G.add_node(ent_text, type=ent_label)
        G.add_edge(user, ent_text, relation="review")  # Hubungkan user dengan entitas

# Visualisasi Knowledge Graph
net = Network(notebook=True, height="800px", width="100%", bgcolor="#222222", font_color="white")
net.from_nx(G)

# Tambahkan fitur interaktif
net.toggle_physics(True)  # Aktifkan fisika agar node tidak tumpang tindih
net.force_atlas_2based(gravity=-50, central_gravity=0.01)  # Sesuaikan gaya tarik antar node

# Simpan ke file HTML dan tampilkan
net.show("knowledge_graph.html")

knowledge_graph.html
